In [ ]:
import traintools

In [ ]:
traintools.DATA_HUB['fra-eng'] = (traintools.DATA_URL + 'fra-eng.zip',
								  '94646ad1522d915e7b0f9296181140edcf86a4f5')

def read_data_nmt():
	data_dir = traintools.download_extract('fra-eng')
	with open(data_dir + '/fra.txt', 'r', encoding='utf-8') as f:
		return f.read()

raw_text = read_data_nmt()
print(raw_text)

In [ ]:
def preprocess_nmt(text):
	def no_space(char,prev_char):
		return char in set(',.!?') and prev_char != ' '
	text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()
	out = []
	for i, char in enumerate(text):
		if i > 0 and no_space(char, text[i-1]):
			out.append(' ' + char)
		else:
			out.append(char)
	return ''.join(out)

text = preprocess_nmt(raw_text)
print(text)

In [ ]:
def tokenize_nmt(text,num_examples=None):
	source = []
	target = []

	for i,line in enumerate(text.split('\n')):
		if num_examples and i > num_examples:
			break
		parts = line.split('\t')
		if len(parts) == 2:
			source.append(parts[0].split(' '))
			target.append(parts[1].split(' '))
	return source, target

source ,target = tokenize_nmt(text)

print(source[:6])
print(target[:6])